# 1. Preparations

In [ ]:
!rm -rf ./logs/
!pip install tensorflow_datasets
!pip install -q -U keras-tuner
!pip install -q pyyaml h5py
!spacy download de_core_news_md

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import urllib, IPython, os, datetime, re, nltk, tensorboard, operator, json, requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout, BatchNormalization 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.activations as activations
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
import kerastuner as kt
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from collections import Counter
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups

%matplotlib inline
%load_ext tensorboard

**Google Cobalb runtime needs to be restarted before Spacy can be used!**

In [ ]:
os.kill(os.getpid(), 9)

Additional downloads

In [ ]:
nltk.download('punkt')
spacy.prefer_gpu()
nlp = spacy.load('de_core_news_md')

In [ ]:
session_id = datetime.datetime.now().strftime("%d/%m/%Y-%H:%M")

Add the file names which should be processed. Files should be named after philosophers (e.g. `plato.txt`).

In [ ]:
file_names = [
    'kant.txt', 
    'nietzsch.txt', 
    'platon.txt', 
    'rousseau.txt']
phil_names = [name[:-4].capitalize() for name in file_names]

# 2. Loading data and handle stop words

## 2.1. Loading files with Keras

In [ ]:
prefix = 'file://'
processed_path = '/content/drive/My Drive/RUAK/input/processed/'
url = urllib.parse.quote(processed_path)

for file_name in file_names:
  text_dir = tf.keras.utils.get_file(file_name, origin=prefix+url+file_name)

parent_dir = os.path.dirname(text_dir)

## 2.2 Load [stop words](https://en.wikipedia.org/wiki/Stop_word)

In [87]:
def replace_umlaut(string):
    string = string.replace('ä', 'ae')
    string = string.replace('ö', 'oe')
    string = string.replace('ü', 'ue')
    string = string.replace('Ä', 'Ae')
    string = string.replace('Ö', 'Oe')
    string = string.replace('Ü', 'Ue')
    string = string.replace('ß', 'ss')
    return string

response = json.loads(requests.get('https://countwordsfree.com/stopwords/german/json').text)
stop_words = set([replace_umlaut(word) for word in response['words']])
print(f'Loaded {len(stop_words)} stop words.')

Loaded 579 stop words.


Manually add stop words.

In [88]:
manual_stop_words = set(['naemlich', 'bloss'])

stop_words = stop_words.union(manual_stop_words)
print(f'Total stop words count: {len(stop_words)}.')

Total stop words count: 581.


# 3. Collect and organize data

Function to add words to `words_without_stop_words` and `unique_words_without_stop_words`.

In [90]:
def add_words(sentence):
  for word in sentence.split():
    word = re.sub(r"[^a-zA-Z]+", "", word)
    if word == '':
      continue
    if word.lower() not in stop_words:
      words_without_stop_words.append(word)
      unique_words_without_stop_words.add(word)

words_without_stop_words = []
unique_words_without_stop_words = set()

## 3.1. Extract sentences

Extract sentences from files and creates labels list. Adjust the language for the `nltk.sent_tokenizer` if needed.

In [91]:
labels = []
sentences = []

for index, file_name in enumerate(file_names):

  path = os.path.join(parent_dir, file_name)

  with open(path, 'rb') as file: 
    text = str(file.read())
    nltk_sentences = nltk.sent_tokenize(text, language='german')

    for sentence in nltk_sentences:
      if ' ' in sentence == False:
        continue
      if len(sentence) <= 20:
        continue
      sentence = str(sentence).replace("b'", "")
      sentences.append(sentence)
      labels.append(index)
      add_words(sentence)

    print(f"Sentences for {file_name} with label: {index} added.")

print(f'\n{df.shape[0]} sentences found.')
print(f'{len(words_without_stop_words)} words found (excl. stop words).')
print(f'{len(unique_words_without_stop_words)} unique words found (excl. stop words).')

KeyboardInterrupt: ignored

## 3.2. Create DataFrame and extend data

Collect most commen words except the stop words.

In [ ]:
most_common = [word[0] for word in Counter(words_without_stop_words).most_common(20)]
most_common_count = mydict = {k: v for k, v in Counter(words_without_stop_words).most_common(20)}

Some helper methods

In [ ]:
def stop_word_quote_fn(sentence):
  count = 0
  for word in sentence.split():
    word = re.sub(r"[^a-zA-Z]+", "", word)
    if word.lower() in stop_words:
      count += 1
  return round(count/len(sentence.split()) * 100, 2)

def stop_word_count_fn(sentence):
  count = 0
  for word in sentence.split():
    word = re.sub(r"[^a-zA-Z]+", "", word)
    if word.lower() in stop_words:
      count += 1
  return count

def mean_word_length_fn(sentence):
  return round(np.array([len(word) for word in sentence.replace('.','').split()]).mean(), 2)

def pos_count(sentence, pos):
  doc = nlp(sentence)
  return len([w.pos_ for w in doc if w.pos_ == pos])

### 3.2.1. Create DataFrame

In [ ]:
df = pd.DataFrame({'label': labels, 'sentence': sentences})

### 3.2.2. Construct new data

In [ ]:
df['philosopher'] = df['label'].map(lambda x: phil_names[x])
df['word_count'] = df['sentence'].str.split().str.len()
df['mean_word_length'] = df['sentence'].map(mean_word_length_fn)
df['stop_words_quote'] = df['sentence'].map(stop_word_quote_fn)
df['stop_words_count'] = df['sentence'].map(stop_word_count_fn)

Add columns and values for [POS tagging](https://en.wikipedia.org/wiki/Part-of-speech_tagging). Annotations can be found [here](https://spacy.io/api/annotation). **This may take a while!**

In [ ]:
pos_tags = ['ADJ', 'ADV', 'ADP', 'AUX', 'DET', 'NUM', 'X', 'INTJ', 'CCONJ', 'SCONJ', 'PROPN', 'NOUN', 'DET', 'PRON', 'PART', 'VERB']
for pos in pos_tags:
  df[f'{pos}_count'] = df['sentence'].map(lambda sen: pos_count(sen, pos))

### 3.2.3. Preview DataFrame

In [ ]:
df.head(len(sentences))

# 4. Store and load a DataFrame

Save DataFrame to CSV if needed.

In [ ]:
df.to_csv('/content/drive/My Drive/RUAK/output/dataframe/dataframe.csv')

Load the DataFrame from CSV if needed.

In [ ]:
df = pd.read_csv('/content/drive/My Drive/RUAK/output/dataframe/dataframe.csv')

# 5. Visualization of data

## 5.1 Prepare values for visualization

[Lemmatize](https://en.wikipedia.org/wiki/Lemmatisation)

In [ ]:
def lemmatize(word):
  doc = nlp(word)
  for word in doc:
    if word.pos_ == 'VERB':
      return word.lemma_.replace(' ', '') 
    return word.text.replace(' ', '')

Count vocabulary

In [ ]:
def vocabulary_count_fn(series):
  vocabulary = set()
  for sentence in series:
    words = sentence.split()
    for word in words:
      if word.lower() not in stop_words:
        word = re.sub(r"[^a-zA-Z]+", "", word)
        word = lemmatize(word)
        vocabulary.add(word.lower())
  return len(vocabulary)

Values from DataFrame

In [ ]:
median_sentence_length = df.groupby('philosopher')['word_count'].median()
mean_stop_words = df.groupby('philosopher')['stop_words_quote'].mean()
sentence_count = df.groupby('philosopher')['sentence'].count()
unique_vocabulary_count = df.groupby('philosopher')['sentence'].apply(lambda ser: vocabulary_count_fn(ser))

## 5.2. Draw visualization

### 5.2.1. Comparing philosophers

In [ ]:
fig, axs = plt.subplots(4,1, figsize=(10,15))

fig.tight_layout(h_pad=6)

axs[0].bar(phil_names, sentence_count)
axs[0].set_ylabel('Number of sentences', fontdict={'color':'gray', 'size':12})
axs[0].tick_params(axis='both', colors='gray', labelsize=12)
axs[0].grid()

axs[1].bar(phil_names, median_sentence_length)
axs[1].set_ylabel('Median sentence lenth', fontdict={'color':'gray', 'size':12})
axs[1].tick_params(axis='both', colors='gray', labelsize=12)
axs[1].grid()

axs[2].bar(phil_names, unique_vocabulary_count)
axs[2].set_ylabel('Unique vocabulary count\n(excl. stop words)', fontdict={'color':'gray', 'size':12})
axs[2].tick_params(axis='both', colors='gray', labelsize=12)
axs[2].grid()

axs[3].bar(phil_names, mean_stop_words)
axs[3].set_ylabel('Median stop words quote', fontdict={'color':'gray', 'size':12})
axs[3].tick_params(axis='both', colors='gray', labelsize=12)
axs[3].grid()

plt.show()

### 5.2.2. Common words in sentences

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(30,10))

axs.bar(most_common_count.keys(), most_common_count.values(), color='orange')
axs.set_ylabel('Number of sentences', fontdict={'color':'gray', 'size':12})
axs.tick_params(axis='both', colors='gray', labelsize=12)
axs.grid()

plt.show()

### 5.2.3. Word cloud

In [ ]:
wordcloud = WordCloud(width=5000, 
                      height=4000,
                      max_words=20,  
                      background_color ='black', 
                      stopwords = stop_words, 
                      min_font_size = 10).generate_from_frequencies(most_common_count) 

plt.figure(figsize=(20, 12), facecolor='k', edgecolor ='k') 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad=0) 
plt.show() 

# 6. Prepare text data

## 6.1. Tokenize

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['sentence'].values)
print(f"{len(df['sentence'].values)} sentences from {len(file_names)} authors.")
print(f'{len(tokenizer.word_counts)} unique vocabularies.')

## 6.2. Encode

In [ ]:
encoded_sentences = tokenizer.texts_to_sequences(df['sentence'].values)
padded_sentences = pad_sequences(encoded_sentences, padding='post')

Test the encoder

In [ ]:
print(df['sentence'].values[7])
print(np.array(padded_sentences[7]))
print(tokenizer.sequences_to_texts([padded_sentences[7]]))

## 6.3. Splitting

Create train and test data for the fitting proccess.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(padded_sentences, df['label'].values, test_size=0.1)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

# 7. Hyperparameter tuning

## 7.1. Setup the hypermodel

Load the Word2Vec model (700 dims, 100 epochs, window 7) for providing the weights for the embedding layer.

In [ ]:
model_path = os.path.abspath("/content/drive/My Drive/RUAK/output/embedding/w2v/") # This path may need to be changed.

def get_embedding_matrix(model_name):
    model = Word2Vec.load(f'{model_path}/{model_name}')
    embedding_matrix = np.zeros((len(model.wv.vocab), model.vector_size))
    
    for i in range(len(model.wv.vocab)):
        embedding_vector = model.wv[model.wv.index2word[i]]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print(f"Embedding_matrix shape: {embedding_matrix.shape}")
    return embedding_matrix

embedding_matrix = get_embedding_matrix('full_700_iter100_win7_8.model')

Define the model

In [ ]:
def hypermodel(hp):

  hp_dense_count = hp.Int('dense_count', min_value=1, max_value=7, step=1)
  hp_embedding_trainable = hp.Choice('embedding_trainable', [True, False])
  hp_with_batch_normalization = hp.Choice('with_batch_normalization', [True, False])
  hp_lstm_units = hp.Int('lstm_units', 256, 512, step=128)
  hp_dropout = hp.Choice('dropout', [0.0, 0.1, 0.25])
  hp_learning_rate = hp.Choice('learning_rate', [0.01, 0.001, 0.0001])
  hp_adam_epsilon = hp.Choice('adam_epsilon', values=[1e-07, 1e-08])

  model = tf.keras.Sequential()
                                    
  model.add(Embedding(len(embedding_matrix),
                    output_dim=700,
                    weights=[embedding_matrix], 
                    trainable=hp_embedding_trainable,
                    mask_zero=True))
  
  model.add(Bidirectional(LSTM(hp_lstm_units, return_sequences=True)))
  if hp_embedding_trainable == True:
    BatchNormalization()
  model.add(Dropout(hp_dropout))

  model.add(Bidirectional(LSTM(hp_lstm_units, return_sequences=True)))
  if hp_embedding_trainable == True:
    BatchNormalization()  
  model.add(Dropout(hp_dropout))

  model.add(Bidirectional(LSTM(hp_lstm_units)))
  if hp_embedding_trainable == True:
    BatchNormalization()  
  model.add(Dropout(hp_dropout))

  for i in range(hp_dense_count):
    hp_dense_units = hp.Int(f'dense_units{i}', 64, 512, step=64)
    hp_dense_activation = hp.Choice(f'dense_activation_{i}', values=['tanh', 'relu'])
    model.add(Dense(hp_dense_units, activation=hp_dense_activation))

  model.add(Dense(len(file_names), activation='softmax'))

  model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate, epsilon=hp_adam_epsilon),
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

## 7.2. Run the tuner

Some parameters needed for tuning and training.

In [ ]:
batch_size=40
epochs=1
search_epochs=1
early_stopping_patience=5
executions_per_trial=1
hyperband_iterations=1

The result are the optimal hyperparameters: `best_hps`.

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait=True)

tuner = kt.Hyperband(hypermodel,
                     objective='val_accuracy', 
                     executions_per_trial=executions_per_trial,
                     factor=3,
                     max_epochs=epochs+5,
                     hyperband_iterations=hyperband_iterations,
                     directory='/content/drive/My Drive/RUAK/output/hp_tuning', # This path may need to be changed.
                     project_name='RUAK',
                     overwrite=True)

tuner.search(X_train, y_train, 
             epochs=search_epochs,
             validation_data = (X_valid, y_valid),
             callbacks = [ClearTrainingOutput(), EarlyStopping('val_accuracy', patience=1)],
             verbose=2)

best_hps = tuner.get_best_hyperparameters(1)[0]
best_models = tuner.get_best_models(num_models=3)

print(f"""
Optimal values:
- embedding is trainable {best_hps.get('embedding_trainable')}
- number of units for dense layers {best_hps.get('dense_units')}
- number of units for lstm layers {best_hps.get('lstm_units')}
- learning rate {best_hps.get('learning_rate')}
- dropout rate {best_hps.get('dropout')}
""")

tuner.results_summary()

# 8. Model preparation

Get summaries of the best models and choose the model for training.

In [ ]:
for model in best_models:
  model.summary()

Choose preferred model

In [ ]:
chosen_model = best_models[0]

Plot model structure

In [ ]:
plot_model(chosen_model, show_shapes=True, show_layer_names=True)

## 8.1. Prepare callbacks

TensorBoard preparations

In [ ]:
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%d/%m/%Y - %H:%M"))
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Prepare `ModelCheckpoint` and `EarlyStopping` callbacks.

In [ ]:
checkpoint_path = f"/content/drive/My Drive/RUAK/output/training_checkpoints/{session_id}/cp.ckpt" # This path may need to be changed.
checkpoint_dir = os.path.dirname(checkpoint_path)

model_path = f"/content/drive/My Drive/RUAK/output/models/model-{session_id}.h5" # This path may need to be changed.

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

es_callback = EarlyStopping('val_accuracy', patience=early_stopping_patience, restore_best_weights=True)

## 8.2. Model training

In [ ]:
 history = chosen_model.fit(X_train, 
                            y_train, 
                            epochs=epochs, 
                            batch_size=batch_size, 
                            validation_data=(X_valid, y_valid), 
                            callbacks=[cp_callback, es_callback, tb_callback])

# 9. Save or load the model

## 9.1. Save the model

In [ ]:
model.save('/content/drive/My Drive/RUAK/output/models/phil_model.h5') # This path may need to be changed.

## 9.2. Load the model

In [ ]:
chosen_model = tf.keras.models.load_model('/content/drive/My Drive/RUAK/output/models/phil_model.h5') # This path may need to be changed.

## 9.3. Load stored weights

`{ID}` needs to be provided. Should be some kind of `session_id`.

In [ ]:
checkpoint_id = # Place the `session_id`

In [ ]:
checkpoint_path = f"/content/drive/My Drive/RUAK/training_checkpoints/{checkpoint_id}/cp.ckpt" # This path may need to be changed.
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
chosen_model.load_weights(latest)

# 10. Evaluation

## 10.1. Show loss and accuracy

In [ ]:
test_loss, test_acc = chosen_model.evaluate(valid_data)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

## 10.2. Test the model

Test the model. Add a `sample_sentence` to get the probability distribution for each author.

In [ ]:
sample_sentence = "Die Vernunft ist staerker als der Wille."

In [ ]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sample_text, pad):
  encoded_sample_text = encoder.encode(sample_text)
  if pad:
    encoded_sample_text = pad_to_size(encoded_sample_text, batch_size)
  encoded_sample_text = tf.cast(encoded_sample_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_text, 0))
  return (predictions)

predictions = sample_predict(sample_sentence_text, pad=True)

print(predictions)

# 11. TensorBoard analysis

In [ ]:
%tensorboard --logdir logs